Features: lemma, pos, case, voice, verb form, head, deprel.

In [1]:
from conllu.parser import parse
from collections import OrderedDict
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

In [2]:
s = open("C:/Users/1/Desktop/train.txt", "r", encoding="utf-8")
z = open("C:/Users/1/Desktop/test.txt", "r", encoding="utf-8")
s = s.read()
z = z.read()
train = parse(s)
test = parse(z)

In [3]:
train[0][0]

OrderedDict([('id', 1),
             ('form', 'Kävelyreitti'),
             ('lemma', 'kävely#reitti'),
             ('upostag', 'NOUN'),
             ('xpostag', 'N'),
             ('feats', OrderedDict([('Case', 'Nom'), ('Number', 'Sing')])),
             ('head', 0),
             ('deprel', 'root'),
             ('deps', None),
             ('misc', None)])

In [4]:
def get_feature(dic, col_name, array):
    for item in dic.items():
        if item[0] == col_name:
            if item[1] == None:
                return array.append("No")
            else:
                return array.append(item[1])

In [5]:
def get_feature_from_feats(dic, feat_name, array):
    for item in dic.items():
        if item[0] == "feats":
            if item[1] == None:
                return array.append("No")
            else:
                dic_feats = dict(item[1])
                for feat in dic_feats.items():
                    if feat[0] == feat_name:
                        return array.append(feat[1])
                    else:
                        return array.append("No")

In [6]:
def get_matrix_and_deprels(data):
    matrix = []
    deprels = []
    
    for i in range(len(data)):
        for x in range(len(data[i])):
            dic = dict(data[i][x])
        
            arr = []
            lemma = get_feature(dic, "lemma", arr)
            upostag = get_feature(dic, "upostag", arr)
            case = get_feature_from_feats(dic, "Case", arr)
            voice = get_feature_from_feats(dic, "Voice", arr)
            verb_form = get_feature_from_feats(dic, "VerbForm", arr)
            head = get_feature(dic, "head", arr)
            deprel = get_feature(dic, "deprel", arr)
            matrix.append(arr)
            
            arr2 = []
            deps = get_feature(dic, "deps", arr2)
            deprels.append(arr2)
                        
    return matrix, deprels

In [7]:
matrix_train = get_matrix_and_deprels(train)[0]
matrix_test = get_matrix_and_deprels(test)[0]
deprels_train = get_matrix_and_deprels(train)[1]
deprels_test = get_matrix_and_deprels(test)[1]

In [8]:
matrix_train[0]

['kävely#reitti', 'NOUN', 'Nom', 'No', 'No', 0, 'root']

In [9]:
le = preprocessing.LabelEncoder()

matrix_train_to_df = pd.DataFrame(matrix_train)
matrix_train_to_df_new = matrix_train_to_df.apply(le.fit_transform)
deprels_train_to_df = pd.DataFrame(deprels_train)
deprels_train_to_df_new = deprels_train_to_df.apply(le.fit_transform)
matrix_test_to_df = pd.DataFrame(matrix_test)
matrix_test_to_df_new = matrix_test_to_df.apply(le.fit_transform)
deprels_test_to_df = pd.DataFrame(deprels_test)
deprels_test_to_df_new = deprels_test_to_df.apply(le.fit_transform)

In [10]:
clf = DecisionTreeClassifier()
clf = clf.fit(matrix_train_to_df_new, deprels_train_to_df_new)
predictions = clf.predict(matrix_test_to_df_new)